In [2]:
import detect_peaks as dp
import nanonis
import distributions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from matplotlib import rc
from scipy.signal import savgol_filter
import glob
import colorcet as cc
import pickle
from lmfit.models import LorentzianModel, ConstantModel, GaussianModel
plt.rcParams.update({'font.size': 20})
plt.rcParams["figure.figsize"] = [8,6]
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Helvetica']})
plt.switch_backend('Qt5Agg')
%matplotlib qt
spectra = nanonis.biasSpectroscopy()

def loadSpectra(filename, normTo, biasoff, color, label, Voff,i):
    spectra = nanonis.biasSpectroscopy()
    spectra.load(filename)
    spectra.normalizeTo(normTo)
    spectra.biasOffset(biasoff)
    ax.plot(spectra.bias*1e3,spectra.conductance + Voff*i, color=color, label=label)

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL) 
def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
#legend
#r_ is the measurement point
#r__ is the position list of the atoms
#J_ is the exchange of each atom with the SC
#theta_ is the spin orientation of each atom
#G0 is the GF for the BCS
#G0_ is the GF for the atoms

class spin5:

    def __init__(self,N):
        self.N = N
        self.s0 = np.array([[1,0],[0,1]])
        self.s1 = np.array([[0,1],[1,0]])
        self.s3 = np.array([[1,0],[0,-1]])
        self.theta_ = np.zeros(N)
        self.J_ = [0.4]
        self.U_=[0.2]
        self.r__ = [(0,0)]


    def G0(self,r0,r1,E):
        delta = 0.01
        w = np.sqrt(delta**2-E**2)
        x = np.sqrt((r1[0]-r0[0])**2+(r1[1]-r0[1])**2) #distance between atoms
        r=(1+1j*w)*x
        J0=scisp.jv(0,r) #bessel function 0th order u=argument
        H0=mpmath.struveh(0,r) #gives the struve function  
        BCS = E/w*np.kron(self.s0,self.s0) + delta/w*np.kron(self.s1,self.s0)
        return  BCS * np.real(J0+1j*H0) -1j*np.kron(self.s3,self.s0) * np.imag(J0 + 1j*H0)
    
    def V(self,theta,J,U):
        return J * np.cos(theta)*np.kron(self.s0,self.s3) + J*np.sin(theta)*np.kron(self.s0,self.s1)+U*np.kron(self.s3,self.s0)
    
    def M(self,E):
        n=self.N
        M = np.zeros((4*n,4*n),dtype=complex)
        for i in range(0,4*n,4):
            for j in range(0,4*n,4):
                M[i:i+4,j:j+4] = np.dot(self.G0(self.r__[i//4],self.r__[j//4],E),self.V(self.theta_[j//4],self.J_[j//4],self.U_[j//4]))
        return M

    def G0_(self,r_,E): #r_ is the measurement point
        G0_ = np.zeros((4*self.N,4),dtype=complex)
        n=0
        for i in range(0,4*self.N,4):
            G0_[i:i+4,0:4] = self.G0(self.r__[n],r_,E)
            n+=1
        return G0_
    
    def G(self,r_,E):
        MM = np.linalg.inv(np.identity(self.N*4)-self.M(E))
        GG = np.dot(MM,self.G0_(r_,E))
        G = self.G0((0,0),(0,0),E)
        n=0
        for i in range(0,4*self.N,4):
            G += np.dot(np.dot( self.G0(r_,self.r__[n],E) , self.V(self.theta_[n] ,self.J_[n],self.U_[n])), GG[i:i+4,0:4] )
            n+=1
        return G

    def DOS(self,r_,E):
        return np.imag(np.trace(self.G(r_,E)))
    
    def ElecDOS(self,r_,E):
        return np.imag(np.trace(np.dot(self.G(r_,E),np.diag((1,1,0,0)))))

    def HoleDOS(self,r_,E):
        return np.imag(np.trace(np.dot(self.G(r_,E),np.diag((0,0,1,1)))))

    def BCS(self,E,delta):
        w = np.sqrt(delta**2-E**2)
        A = E/w + delta/w
        return A


In [ ]:
E = np.linspace(-0.02,0.02,200)
sim=spin5(1)

v1 = []

for i in E:
    v1.append(np.sign(i)*sim.DOS((3,0),i+0.0001j*np.sign(i)))


plt.plot(E,v1)


# Square

In [248]:
class spin5_s:

    def __init__(self,N):
        self.N = N
        self.s0 = np.array([[1,0],[0,1]])
        self.s1 = np.array([[0,1],[1,0]])
        self.s3 = np.array([[1,0],[0,-1]])
        self.theta_ = np.zeros(N)#[0.0,np.pi,np.pi,np.pi,np.pi]
        self.J_ = [0.03,0.03,0.03,0.03]
        self.U_=[0,0,0,0]
        self.r__ = [(0,0),(-1,2),(-2,-1),(1,-2)]



    def G0(self,r1,r2,E):
        x1=r1[0]-r2[0]
        x2=r1[1]-r2[1]
        delta=0.78*1e-3/1.03
        m=20.956
        pf=0.274
        w = np.sqrt(delta**2-E**2)
        BCS = E/w*np.kron(self.s0,self.s0) + delta/w*np.kron(self.s1,self.s0)
        xi=np.kron(self.s3,self.s0)
        if np.abs(x1)<0.00001 and np.abs(x2)<0.00001:
                G1=(2*m/np.pi)*BCS
                G2=0.0
        elif np.abs(x2)<0.00001:
            G1=(np.exp(-(m*w/pf)*np.abs(x1))*( (1/np.abs(x1))*pf*np.sin(pf*(np.abs(x1)))+(pf**2)*np.cos(pf*np.abs(x1)) ) )*(m/(np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*np.abs(x1))*( (1/np.abs(x1))*pf*np.cos(pf*(np.abs(x1)))+(pf**2)*np.sin(pf*np.abs(x1)) ) -pf/np.abs(x1))*(m/(np.pi*pf**2))*xi
        elif np.abs(x1)<0.00001:
            G1=(np.exp(-(m*w/pf)*np.abs(x2))*( (1/np.abs(x2))*pf*np.sin(pf*(np.abs(x2)))+(pf**2)*np.cos(pf*np.abs(x2)) ) )*(m/(np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*np.abs(x2))*( (1/np.abs(x2))*pf*np.cos(pf*(np.abs(x2)))+(pf**2)*np.sin(pf*np.abs(x2)) ) -pf/np.abs(x2))*(m/(np.pi*pf**2))*xi
        elif x1+x2>=0.00001 and x1-x2>=0.00001:
            G1=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.sin(pf*(x1+x2))+np.exp(-(m*w/pf)*(x1-x2))*(1/x1-1/x2)*pf*np.sin(pf*(x1-x2)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.cos(pf*(x1+x2))+np.exp(-(m*w/pf)*(x1-x2))*(1/x1-1/x2)*pf*np.cos(pf*(x1-x2)) -2*pf/x1)*(m/(2*np.pi*pf**2))*xi
        elif x1+x2>0.00001 and x1-x2<0.00001:
            G1=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.sin(pf*(x1+x2))+np.exp(-(m*w/pf)*(x2-x1))*(1/x2-1/x1)*pf*np.sin(pf*(x2-x1)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.cos(pf*(x1+x2))+np.exp(-(m*w/pf)*(x2-x1))*(1/x2-1/x1)*pf*np.cos(pf*(x2-x1)) -2*pf/x2)*(m/(2*np.pi*pf**2))*xi
        elif x1+x2<=0.00001 and x1-x2<=0.00001:
            G1=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.sin(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x1+x2))*(-1/x1+1/x2)*pf*np.sin(pf*(-x1+x2)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.cos(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x1+x2))*(-1/x1+1/x2)*pf*np.cos(pf*(-x1+x2)) +2*pf/x1)*(m/(2*np.pi*pf**2))*xi
        elif x1+x2<0.00001 and x1-x2>0.00001:
            G1=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.sin(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x2+x1))*(-1/x2+1/x1)*pf*np.sin(pf*(-x2+x1)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.cos(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x2+x1))*(-1/x2+1/x1)*pf*np.cos(pf*(-x2+x1)) +2*pf/x2)*(m/(2*np.pi*pf**2))*xi
        return G1+G2
    
    def V(self,theta,J,U):
        return J * np.cos(theta)*np.kron(self.s0,self.s3) + J*np.sin(theta)*np.kron(self.s0,self.s1)+U*np.kron(self.s3,self.s0)
    
    def M(self,E):
        n=self.N
        M = np.zeros((4*n,4*n),dtype=complex)
        for i in range(0,4*n,4):
            for j in range(0,4*n,4):
                M[i:i+4,j:j+4] = np.dot(self.G0(self.r__[i//4],self.r__[j//4],E),self.V(self.theta_[j//4],self.J_[j//4],self.U_[j//4]))
        return M

    def G0_(self,r_,E): #r_ is the measurement point
        G0_ = np.zeros((4*self.N,4),dtype=complex)
        n=0
        for i in range(0,4*self.N,4):
            G0_[i:i+4,0:4] = self.G0(self.r__[n],r_,E)
            n+=1
        return G0_
    
    def G(self,r_,E):
        MM = np.linalg.inv(np.identity(self.N*4)-self.M(E))
        GG = np.dot(MM,self.G0_(r_,E))
        G = self.G0((0,0),(0,0),E)
        n=0
        for i in range(0,4*self.N,4):
            G += np.dot(np.dot( self.G0(r_,self.r__[n],E) , self.V(self.theta_[n] ,self.J_[n],self.U_[n])), GG[i:i+4,0:4] )
            n+=1
        return G

    def DOS(self,r_,E):
        return np.imag(np.trace(self.G(r_,E)))
    
    def ElecDOS(self,r_,E):
        return np.imag(np.trace(np.dot(self.G(r_,E),np.diag((1,1,0,0)))))

    def HoleDOS(self,r_,E):
        return np.imag(np.trace(np.dot(self.G(r_,E),np.diag((0,0,1,1)))))

    def BCS(self,E,delta):
        w = np.sqrt(delta**2-E**2)
        A = E/w + delta/w
        return A


Convolution

In [249]:
#dynes

def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

def dynes_curve(energy,Delta,dynesParameter):
    dynesParameter = complex(0, dynesParameter)
    density = np.sign(energy)*np.real(np.divide(energy-np.sign(energy)*dynesParameter,np.sqrt(np.power(energy-np.sign(energy)*dynesParameter, 2)-np.power(Delta, 2))))
    return density
def fermiDirac (axis, temperature):
    return np.divide(1,np.exp(np.divide(axis,np.multiply(8.617E-5,temperature)))+1)
#convolucion

def dynesConvolute(bias,E,conductance,delta,T,gamma):
    curr = []
    for Vp in bias:
        currp = np.trapz(dynes_curve(E+Vp,delta,gamma)*(conductance)*(fermiDirac(E+ Vp, T)-fermiDirac(E, T)),x=E)
        curr.append(currp)
    return -np.gradient(np.array(curr))

# to call


In [251]:
delta_t=0.6e-3
delta_s=0.7E-3/1.03
gamma_s=50e-6
gamma_t=50e-6
file="C:/Users/jonor/Desktop/PhD/Lanak/BadHonnef/Flowers/Sqrt5_flower/S200622_043.dat"

spectra = nanonis.biasSpectroscopy()
spectra.load(file)
spectra.biasOffset(-0.00004)
E = np.linspace(-4*delta_s,4*delta_s,1000)
sim=spin5_s(4)
conductance = []
for i in E:
    if i>0:
        conductance.append(0.6*(sim.DOS((0,0),i+gamma_s*1j))+12*dynes_curve(i,delta_s,gamma_s))
    else:
        conductance.append((sim.DOS((0,0),i+gamma_s*1j))+12*dynes_curve(i,delta_s,gamma_s))
#dynes
delta_t=0.65e-3
delta_s=0.78E-3/1.03
bias = np.linspace(-3*delta_s,3*delta_s,200)
conv_cond = dynesConvolute(bias,E,conductance,delta_t,1.1,gamma_t)
#plt.plot(E,conductance)
expb=[]
exp=[]
for i in range(find_nearest(spectra.bias,3*delta_s),find_nearest(spectra.bias,-3*delta_s)):
    expb.append(spectra.bias[i])
    exp.append(spectra.conductance[i])
plt.plot(expb,exp/exp[-1])
#plt.plot(spectra.bias,spectra.conductance/spectra.conductance[0])
plt.plot(bias,conv_cond/(conv_cond[0]))
expb3=expb
fitb3=bias
fit3=conv_cond/(conv_cond[0])
exp3=exp/exp[-1]

In [200]:
expb1,fitb1,fit1,exp1
expb2,fitb2,fit2,exp2

In [181]:
E = np.linspace(-4*delta_s,4*delta_s,1000)
sim=spin5_s(5)
conductance = []
for i in E:
    conductance.append(sim.DOS((0,0),i+gamma_s*1j)+15*dynes_curve(i,delta_s,gamma_s))
plt.plot(E,conductance)

In [12]:
plt.imshow(v1,aspect='auto',origin='lower')

Two adatoms angle

In [4]:
class spin5_new:

    def __init__(self,N,J,theta):
        self.N = N
        self.s0 = np.array([[1,0],[0,1]])
        self.s1 = np.array([[0,1],[1,0]])
        self.s3 = np.array([[1,0],[0,-1]])
        self.theta_ = theta
        self.J_ = J
        self.U_=[0,0]
        self.r__ = [(0,0),(1,2)]



    def G0(self,r1,r2,E):
        x1=r1[0]-r2[0]
        x2=r1[1]-r2[1]
        delta=0.01
        m=1
        pf=1
        w = np.sqrt(delta**2-E**2)
        BCS = E/w*np.kron(self.s0,self.s0) + delta/w*np.kron(self.s1,self.s0)
        xi=np.kron(self.s3,self.s0)
        if np.abs(x1)<0.00001 and np.abs(x2)<0.00001:
            G1=(2*m/np.pi)*BCS
            G2=0.0
        elif np.abs(x2)<0.00001:
            G1=(np.exp(-(m*w/pf)*np.abs(x1))*( (1/np.abs(x1))*pf*np.sin(pf*(np.abs(x1)))+(pf**2)*np.cos(pf*np.abs(x1)) ) )*(m/(np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*np.abs(x1))*( (1/np.abs(x1))*pf*np.cos(pf*(np.abs(x1)))+(pf**2)*np.sin(pf*np.abs(x1)) ) -pf/np.abs(x1))*(m/(np.pi*pf**2))*xi
        elif np.abs(x1)<0.00001:
            G1=(np.exp(-(m*w/pf)*np.abs(x2))*( (1/np.abs(x2))*pf*np.sin(pf*(np.abs(x2)))+(pf**2)*np.cos(pf*np.abs(x2)) ) )*(m/(np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*np.abs(x2))*( (1/np.abs(x2))*pf*np.cos(pf*(np.abs(x2)))+(pf**2)*np.sin(pf*np.abs(x2)) ) -pf/np.abs(x2))*(m/(np.pi*pf**2))*xi
        elif x1+x2>=0.00001 and x1-x2>=0.00001:
            G1=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.sin(pf*(x1+x2))+np.exp(-(m*w/pf)*(x1-x2))*(1/x1-1/x2)*pf*np.sin(pf*(x1-x2)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.cos(pf*(x1+x2))+np.exp(-(m*w/pf)*(x1-x2))*(1/x1-1/x2)*pf*np.cos(pf*(x1-x2)) -2*pf/x1)*(m/(2*np.pi*pf**2))*xi
        elif x1+x2>0.00001 and x1-x2<0.00001:
            G1=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.sin(pf*(x1+x2))+np.exp(-(m*w/pf)*(x2-x1))*(1/x2-1/x1)*pf*np.sin(pf*(x2-x1)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.cos(pf*(x1+x2))+np.exp(-(m*w/pf)*(x2-x1))*(1/x2-1/x1)*pf*np.cos(pf*(x2-x1)) -2*pf/x2)*(m/(2*np.pi*pf**2))*xi
        elif x1+x2<=0.00001 and x1-x2<=0.00001:
            G1=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.sin(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x1+x2))*(-1/x1+1/x2)*pf*np.sin(pf*(-x1+x2)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.cos(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x1+x2))*(-1/x1+1/x2)*pf*np.cos(pf*(-x1+x2)) +2*pf/x1)*(m/(2*np.pi*pf**2))*xi
        elif x1+x2<0.00001 and x1-x2>0.00001:
            G1=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.sin(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x2+x1))*(-1/x2+1/x1)*pf*np.sin(pf*(-x2+x1)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.cos(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x2+x1))*(-1/x2+1/x1)*pf*np.cos(pf*(-x2+x1)) +2*pf/x2)*(m/(2*np.pi*pf**2))*xi
        return G1+G2
    
    def V(self,theta,J,U):
        return J * np.cos(theta)*np.kron(self.s0,self.s3) + J*np.sin(theta)*np.kron(self.s0,self.s1)+U*np.kron(self.s3,self.s0)
    
    def M(self,E):
        n=self.N
        M = np.zeros((4*n,4*n),dtype=complex)
        for i in range(0,4*n,4):
            for j in range(0,4*n,4):
                M[i:i+4,j:j+4] = np.dot(self.G0(self.r__[i//4],self.r__[j//4],E),self.V(self.theta_[j//4],self.J_[j//4],self.U_[j//4]))
        return M

    def G0_(self,r_,E): #r_ is the measurement point
        G0_ = np.zeros((4*self.N,4),dtype=complex)
        n=0
        for i in range(0,4*self.N,4):
            G0_[i:i+4,0:4] = self.G0(self.r__[n],r_,E)
            n+=1
        return G0_
    
    def G(self,r_,E):
        MM = np.linalg.inv(np.identity(self.N*4)-self.M(E))
        GG = np.dot(MM,self.G0_(r_,E))
        G = self.G0((0,0),(0,0),E)
        n=0
        for i in range(0,4*self.N,4):
            G += np.dot(np.dot( self.G0(r_,self.r__[n],E) , self.V(self.theta_[n] ,self.J_[n],self.U_[n])), GG[i:i+4,0:4] )
            n+=1
        return G

    def DOS(self,r_,E):
        return np.imag(self.G(r_,E))
    
    def ElecDOS(self,r_,E):
        return np.imag(np.trace(np.dot(self.G(r_,E),np.diag((1,1,0,0)))))

    def HoleDOS(self,r_,E):
        return np.imag(np.trace(np.dot(self.G(r_,E),np.diag((0,0,1,1)))))

    def BCS(self,E,delta):
        w = np.sqrt(delta**2-E**2)
        A = E/w + delta/w
        return A


In [42]:
#the vector of the pseudo spin will be defined as S=SCos0 uz+ Ssin0 ux, we will take S=1. hbar is absorbed by J.
#S.sigma=cos0sigma_z+sin0sigma_x
#We will assume J1=J2 and no ph asymetry
#r1=0 and theta0=0

#We need to calculate J1(rho11-rho22)+J2cos0(rho11-rho22)+J2sin0(rho12+rho21) 
#then we integrate in w and J. From -4Delta to 4Delta and from 0 to J. 

theta=np.linspace(0,2*np.pi,100)
J=np.linspace(0,0.4,20)
w=np.linspace(0.0,0.04,100)
y=[]
for i in theta:
    print(i)
    for j in J:
        sim=spin5_new(2,[j,j],[0,i])
        b=[]
        a=0
        for k in w:
            rho1=sim.DOS((0,0),k+0.0001j)
            rho2=sim.DOS((1,2),k+0.0001j)
            a+=j*(rho1[0,0]-rho1[1,1])+j*np.cos(i)*(rho2[0,0]-rho2[1,1])+j*np.sin(i)*(rho2[0,1]+rho2[1,0]) 
        b.append(a)
    y.append(sum(b))

plt.plot(theta,y)

0.0
0.06346651825433926
0.12693303650867852
0.1903995547630178
0.25386607301735703
0.3173325912716963
0.3807991095260356
0.4442656277803748
0.5077321460347141
0.5711986642890533
0.6346651825433925
0.6981317007977318
0.7615982190520711
0.8250647373064104
0.8885312555607496
0.9519977738150889
1.0154642920694281
1.0789308103237674
1.1423973285781066
1.2058638468324459
1.269330365086785
1.3327968833411243
1.3962634015954636
1.4597299198498028
1.5231964381041423
1.5866629563584815
1.6501294746128208
1.71359599286716
1.7770625111214993
1.8405290293758385
1.9039955476301778
1.967462065884517
2.0309285841388562
2.0943951023931957
2.1578616206475347
2.221328138901874
2.284794657156213
2.3482611754105527
2.4117276936648917
2.475194211919231
2.53866073017357
2.6021272484279097
2.6655937666822487
2.729060284936588
2.792526803190927
2.8559933214452666
2.9194598396996057
2.982926357953945
3.0463928762082846
3.1098593944626236
3.173325912716963
3.236792430971302
3.3002589492256416
3.3637254674799806


In [107]:

def double_Integral(xmin, xmax, ymin, ymax, nx, ny, A):

    dS = ((xmax-xmin)/(nx-1)) * ((ymax-ymin)/(ny-1))

    A_Internal = A[1:-1, 1:-1]

    # sides: up, down, left, right
    (A_u, A_d, A_l, A_r) = (A[0, 1:-1], A[-1, 1:-1], A[1:-1, 0], A[1:-1, -1])

    # corners
    (A_ul, A_ur, A_dl, A_dr) = (A[0, 0], A[0, -1], A[-1, 0], A[-1, -1])

    return dS * (np.sum(A_Internal)+ 0.5 * (np.sum(A_u) + np.sum(A_d) + np.sum(A_l) + np.sum(A_r))+ 0.25 * (A_ul + A_ur + A_dl + A_dr))

x=np.linspace(1,5,15)
theta=np.linspace(0,np.pi,10)
J=np.linspace(0,0.4,30)
w=np.linspace(-0.04,0.04,30)
z=[]
for l in x:
    print(l)
    y=[]
    for i in theta:
        c=[]
        for j in J:
            sim=spin5_new(2,[j,j],[0,i])
            b=[]
            for k in w:
                rho1=sim.DOS((0,0),k+0.0001j)
                rho2=sim.DOS((l,0),k+0.0001j)
                b.append(j*(rho1[0,0]-rho1[1,1])+j*np.cos(i)*(rho2[0,0]-rho2[1,1])+j*np.sin(i)*(rho2[0,1]+rho2[1,0]))
            c.append(b)

        y.append(double_Integral(-0.04,0.04,0.0,0.4,30,30,np.array(c)))
    z.append(y)

plt.imshow(z)

1.0
1.2857142857142856
1.5714285714285714
1.8571428571428572
2.142857142857143
2.4285714285714284
2.7142857142857144
3.0
3.2857142857142856
3.571428571428571
3.8571428571428568
4.142857142857142
4.428571428571429
4.7142857142857135
5.0


In [ ]:

def double_Integral(xmin, xmax, ymin, ymax, nx, ny, A):

    dS = ((xmax-xmin)/(nx-1)) * ((ymax-ymin)/(ny-1))

    A_Internal = A[1:-1, 1:-1]

    # sides: up, down, left, right
    (A_u, A_d, A_l, A_r) = (A[0, 1:-1], A[-1, 1:-1], A[1:-1, 0], A[1:-1, -1])

    # corners
    (A_ul, A_ur, A_dl, A_dr) = (A[0, 0], A[0, -1], A[-1, 0], A[-1, -1])

    return dS * (np.sum(A_Internal)+ 0.5 * (np.sum(A_u) + np.sum(A_d) + np.sum(A_l) + np.sum(A_r))+ 0.25 * (A_ul + A_ur + A_dl + A_dr))

def f(x,T):
    return 1/(np.exp(x/T)+1)

theta=np.linspace(0,np.pi,30)
A=np.linspace(0,0.4,30)
w=np.linspace(-0.04,0.04,30)
x=np.linspace(0,10,30)
y=[]
for i in theta:
    c=[]
    print(i)
    for j in A:
        sim=spin5_new(2,[j,j],[0,i])
        b=[]
        for k in w:
            rho1=sim.DOS((0,0),k+0.0001j)
            rho2=sim.DOS((2,0),k+0.0001j)
            b.append((0.4*(rho1[0,0]-rho1[1,1])+0.4*np.cos(i)*(rho2[0,0]-rho2[1,1])+0.4*np.sin(i)*(rho2[0,1]+rho2[1,0])))
        c.append(b)
    y.append(double_Integral(-0.04,0.04,0.0,0.4,30,30,np.array(c)))
print(y)
plt.plot(theta,y)

In [98]:
plt.plot(x,np.array(Q)[:,0])

plt.plot(x,np.array(Q)[:,1])
plt.plot(x,np.array(Q)[:,2])



Note: you may need to restart the kernel to use updated packages.


CHAINS

In [252]:
class spin5_s:

    def __init__(self,N,a):
        self.N = N
        self.s0 = np.array([[1,0],[0,1]])
        self.s1 = np.array([[0,1],[1,0]])
        self.s3 = np.array([[1,0],[0,-1]])
        self.theta_ = np.zeros(N)#[0.0,np.pi,np.pi,np.pi,np.pi]
        self.J_ = [0.025]*N
        self.U_=np.zeros(N)
        self.r__ = a



    def G0(self,r1,r2,E):
        x1=r1[0]-r2[0]
        x2=r1[1]-r2[1]
        delta=0.78*1e-3/1.03
        m=20.956
        pf=0.274
        w = np.sqrt(delta**2-E**2)
        BCS = E/w*np.kron(self.s0,self.s0) + delta/w*np.kron(self.s1,self.s0)
        xi=np.kron(self.s3,self.s0)
        if np.abs(x1)<0.00001 and np.abs(x2)<0.00001:
            G1=(2*m/np.pi)*BCS
            G2=0.0
        elif np.abs(x2)<0.00001:
            G1=(np.exp(-(m*w/pf)*np.abs(x1))*( (1/np.abs(x1))*pf*np.sin(pf*(np.abs(x1)))+(pf**2)*np.cos(pf*np.abs(x1)) ) )*(m/(np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*np.abs(x1))*( (1/np.abs(x1))*pf*np.cos(pf*(np.abs(x1)))+(pf**2)*np.sin(pf*np.abs(x1)) ) -pf/np.abs(x1))*(m/(np.pi*pf**2))*xi
        elif np.abs(x1)<0.00001:
            G1=(np.exp(-(m*w/pf)*np.abs(x2))*( (1/np.abs(x2))*pf*np.sin(pf*(np.abs(x2)))+(pf**2)*np.cos(pf*np.abs(x2)) ) )*(m/(np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*np.abs(x2))*( (1/np.abs(x2))*pf*np.cos(pf*(np.abs(x2)))+(pf**2)*np.sin(pf*np.abs(x2)) ) -pf/np.abs(x2))*(m/(np.pi*pf**2))*xi
        elif x1+x2>=0.00001 and x1-x2>=0.00001:
            G1=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.sin(pf*(x1+x2))+np.exp(-(m*w/pf)*(x1-x2))*(1/x1-1/x2)*pf*np.sin(pf*(x1-x2)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.cos(pf*(x1+x2))+np.exp(-(m*w/pf)*(x1-x2))*(1/x1-1/x2)*pf*np.cos(pf*(x1-x2)) -2*pf/x1)*(m/(2*np.pi*pf**2))*xi
        elif x1+x2>0.00001 and x1-x2<0.00001:
            G1=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.sin(pf*(x1+x2))+np.exp(-(m*w/pf)*(x2-x1))*(1/x2-1/x1)*pf*np.sin(pf*(x2-x1)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp(-(m*w/pf)*(x1+x2))*(1/x1+1/x2)*pf*np.cos(pf*(x1+x2))+np.exp(-(m*w/pf)*(x2-x1))*(1/x2-1/x1)*pf*np.cos(pf*(x2-x1)) -2*pf/x2)*(m/(2*np.pi*pf**2))*xi
        elif x1+x2<=0.00001 and x1-x2<=0.00001:
            G1=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.sin(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x1+x2))*(-1/x1+1/x2)*pf*np.sin(pf*(-x1+x2)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.cos(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x1+x2))*(-1/x1+1/x2)*pf*np.cos(pf*(-x1+x2)) +2*pf/x1)*(m/(2*np.pi*pf**2))*xi
        elif x1+x2<0.00001 and x1-x2>0.00001:
            G1=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.sin(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x2+x1))*(-1/x2+1/x1)*pf*np.sin(pf*(-x2+x1)) )*(m/(2*np.pi*pf**2))*BCS
            G2=(np.exp((m*w/pf)*(x1+x2))*(-1/x1-1/x2)*pf*np.cos(pf*(-x1-x2))+np.exp(-(m*w/pf)*(-x2+x1))*(-1/x2+1/x1)*pf*np.cos(pf*(-x2+x1)) +2*pf/x2)*(m/(2*np.pi*pf**2))*xi
        return G1+G2
    
    def V(self,theta,J,U):
        return J * np.cos(theta)*np.kron(self.s0,self.s3) + J*np.sin(theta)*np.kron(self.s0,self.s1)+U*np.kron(self.s3,self.s0)
    
    def M(self,E):
        n=self.N
        M = np.zeros((4*n,4*n),dtype=complex)
        for i in range(0,4*n,4):
            for j in range(0,4*n,4):
                M[i:i+4,j:j+4] = np.dot(self.G0(self.r__[i//4],self.r__[j//4],E),self.V(self.theta_[j//4],self.J_[j//4],self.U_[j//4]))
        return M

    def G0_(self,r_,E): #r_ is the measurement point
        G0_ = np.zeros((4*self.N,4),dtype=complex)
        n=0
        for i in range(0,4*self.N,4):
            G0_[i:i+4,0:4] = self.G0(self.r__[n],r_,E)
            n+=1
        return G0_
    
    def G(self,r_,E):
        MM = np.linalg.inv(np.identity(self.N*4)-self.M(E))
        GG = np.dot(MM,self.G0_(r_,E))
        G = self.G0((0,0),(0,0),E)
        n=0
        for i in range(0,4*self.N,4):
            G += np.dot(np.dot( self.G0(r_,self.r__[n],E) , self.V(self.theta_[n] ,self.J_[n],self.U_[n])), GG[i:i+4,0:4] )
            n+=1
        return G

    def DOS(self,r_,E):
        return np.imag(np.trace(self.G(r_,E)))
    
    def ElecDOS(self,r_,E):
        return np.imag(np.trace(np.dot(self.G(r_,E),np.diag((1,1,0,0)))))

    def HoleDOS(self,r_,E):
        return np.imag(np.trace(np.dot(self.G(r_,E),np.diag((0,0,1,1)))))

    def BCS(self,E,delta):
        w = np.sqrt(delta**2-E**2)
        A = E/w + delta/w
        return A


In [267]:
delta_t=0.6e-3
delta_s=0.7E-3/1.03
gamma_s=50e-7
gamma_t=50e-7
file="C:/Users/jonor/Desktop/PhD/Lanak/BadHonnef/Flowers/Sqrt5_flower/S200622_043.dat"

spectra = nanonis.biasSpectroscopy()
spectra.load(file)
spectra.biasOffset(-0.00004)
E = np.linspace(-2*delta_s,2*delta_s,1000)
a=[(0,0),(1,2),(-1,-2),(2,4),(-2,-4),(3,6),(-3,-6),(4,8),(-4,-8)]
sim=spin5_s(9,a)
conductance = []
for i in E:
    conductance.append(sim.DOS((0,0),i+0.00001*1j))
plt.plot(E,conductance)

cond9=conductance

In [272]:
plt.plot(E,np.divide(cond1,1000)+1)
plt.plot(E,np.divide(cond2,1000)+2)
plt.plot(E,np.divide(cond3,1000)+3)
plt.plot(E,np.divide(cond4,1000)+4)
plt.plot(E,np.divide(cond5,1000)+5)
plt.plot(E,np.divide(cond6,1000)+6)
plt.plot(E,np.divide(cond7,1000)+7)
plt.plot(E,np.divide(cond8,1000)+8)
plt.plot(E,np.divide(cond9,1000)+9)


[1, 1, 1, 1, 1]